In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import torch
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [14]:
p = Path()
p

WindowsPath('.')

In [4]:
p.absolute()

WindowsPath('c:/kdt03_deeplearning')

In [ ]:
p2 = p / 'data' / 'fruit'
p2.absolute()

WindowsPath('c:/kdt03_deeplearning/data/fruit')

In [ ]:
for i in p2.iterdir():
    print(list(i.glob('**/*.jpg')))

[]
[WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_0.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_1.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_10.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_100.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_101.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_102.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_103.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_104.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_105.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_106.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_107.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_108.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_109.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_11.jpg'), WindowsPath('data/fruit/Apple Braeburn/Apple Braeburn_110.jpg'), WindowsPath('data/fruit/App

In [ ]:
# 1. 이미지 경로를 설정
# 2. 디렉토리 이름을 별도로 가져와야 함
# 3. 각 디렉토리에 맞춰서 이미지를 관리해야 함
# 4. 이미지를 텐서로 변경해야 함(레이블이 추가되어야 함)
# 5. 전체 이미지를 7:2:1(학습, 검증, 테스트) 데이터로 나눠야 함

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')

In [ ]:
def load_data(root_dir):
    root_path = Path(root_dir)

    # 클래스 레이블 생성
    classes = sorted([d.name for d in root_path.iterdir() if d.is_dir()])   # 클래스명
    class_to_idx = {cls_name:idx for idx, cls_name in enumerate(classes)}   # 레이블 -> 숫자
    idx_to_class = {idx:cls_name for idx, cls_name in enumerate(classes)}

    # 이미지 파일 수집
    images = []
    labels = []
    for cls_name in classes:
        cls_dir = root_path / cls_name
        for img_path in cls_dir.glob('*.jpg'):
            images.append(img_path)
            labels.append(class_to_idx[cls_name])
            
    return images, labels, class_to_idx, idx_to_class

In [48]:
class FruitDataset(Dataset):
    # 과일 이미지 데이터
    def __init__(self, images, labels, transform=None):
        super().__init__()
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = self.images[index]
        label = self.labels[index]

        image = Image.open(img_path).convert('RGB')

        if self.transfrom:
            image = self.transfrom(image)
            
        return image, label

In [34]:
def split_data(images, labels, train_ratio=0.8):
    n_samples = len(images)
    n_train = int(n_samples * train_ratio)

    indices = np.random.permutation(n_samples)
    train_indicies = indices[:n_train]
    val_indicies = indices[n_train:]

    train_images = [images[i] for i in train_indicies]
    val_images = [images[i] for i in val_indicies]
    train_labels = [labels[i] for i in train_indicies]
    val_labels = [labels[i] for i in val_indicies]

    return train_images, val_images, train_labels, val_labels

In [37]:
def get_transForms(image_size=64, augment=True):
    if augment:
        transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    return transform

In [42]:
root_dir = 'data/fruit' # 데이터 위치
num_epochs = 3  # 반복
batch_size = 32 # 학습 크기
image_size = 64

# 재현성을 위한 랜덤 값
torch.manual_seed(42)  
np.random.seed(42)

device = get_device()   # 계산
print(f'{device}를 사용합니다.')

cpu를 사용합니다.


In [49]:
images, labels, class_to_idx, idx_to_class = load_data(root_dir)
train_images, val_images, train_labels, val_labels = split_data(images, labels)

transform_train = get_transForms(image_size, augment=True)
transform_val = get_transForms(image_size, augment=False)

train_dataset = FruitDataset(train_images, train_labels, transform_train)
val_dataset = FruitDataset(val_images, val_labels, transform_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# 파이토치 2.9
## 활성화 함수: 신경망, 뉴런, "비선형성"을 부여, 복잡한 패턴을 학습하는 데 도움
- 1세대(고전): sigmoid, tanh => 속도 좀 느림, 기울기 소실
- 2세대(ReLU): ReLU => 기울기 소실 문제를 해결
- 3세대(확률): GELU(트랜스포머) => 언어 모델에서 표준, 입력에 따른 가중치 부여
- 4세대(진화): Mish(Swish)
-----
> 은닉층
- Tanh: RNN/LSTM에 주로 사용
- ReLU: CNN에 주로 사용
- GELU: 트랜스포머에 주로 사용
-----
> 출력층
- 이진 분류의 출력층: sigmoid
- 다중 분류의 출력층: softmax
-----
> 손실함수: 예측값(y_hat)과 실제값(label) 간의 차이를 측정
- 1세대: 고전 -> MSE
- 2세대: 기계 -> Cross Entropy
- 3세대: 딥러닝 -> Focal, Label Smoothing, Triplet Loss
-----
- 다중 클래스: Cross Entropy
- 이진 분류: BCEwithLogitsLoss
- 회귀: MSELoss

In [ ]:
# 14장에 나오는 CNN으로 가면 됨
import torch.nn as nn
class SimpleCNN(nn.Module):
    pass